# Week 2: Gradio Talking Models

Advanced Gradio interfaces for multi-model conversations

## Step 1: Import Libraries and Setup

In [1]:
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr

openai = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama', 
)

## Step 2: Define Models and Personalities

In [2]:
mistral_model = 'mistral'
llama_model = 'llama3.2:1b'

mistral_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

llama_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

## Step 3: Conversation Management Functions

In [3]:
def run_conversation(initial_message, num_turns=3):
    llama_msgs = [initial_message]
    mistral_msgs = [initial_message]
    
    output = f"# Conversation\n\n"
    output += f"**Llama:** {llama_msgs[0]}\n\n"
    output += f"**Mistral:** {mistral_msgs[0]}\n\n"
    
    for turn in range(num_turns):
        # Llama's response
        llama_context = [{"role": "system", "content": llama_system}]
        for llm, mist in zip(llama_msgs, mistral_msgs):
            llama_context.append({"role": "assistant", "content": llm})
            llama_context.append({"role": "user", "content": mist})
        
        llama_response = openai.chat.completions.create(
            model=llama_model, messages=llama_context
        ).choices[0].message.content
        
        output += f"**Llama:** {llama_response}\n\n"
        llama_msgs.append(llama_response)
        
        # Mistral's response
        mistral_context = [{"role": "system", "content": mistral_system}]
        for llm, mist in zip(llama_msgs, mistral_msgs):
            mistral_context.append({"role": "user", "content": llm})
            mistral_context.append({"role": "assistant", "content": mist})
        mistral_context.append({"role": "user", "content": llama_msgs[-1]})
        
        mistral_response = openai.chat.completions.create(
            model=mistral_model, messages=mistral_context
        ).choices[0].message.content
        
        output += f"**Mistral:** {mistral_response}\n\n"
        mistral_msgs.append(mistral_response)
    
    return output

## Step 4: Create Gradio Interface

In [4]:
input_text = gr.Textbox(
    label='Opening Message',
    info='What should the models talk about?',
    lines=5,
    placeholder='Type a topic or opening message...'
)

turns_slider = gr.Slider(
    minimum=1,
    maximum=10,
    step=1,
    value=3,
    label='Number of Turns',
    info='How many times should each model respond?'
)

output_markdown = gr.Markdown(label='Conversation')

examples = [
    ["Hello, how are you today?", 3],
    ["What do you think about artificial intelligence?", 3],
    ["Is climate change real?", 3],
    ["What's the best way to learn?", 3],
]

interface = gr.Interface(
    fn=run_conversation,
    title='Two Model Conversation',
    description='Watch an argumentative model (Mistral) debate with a polite model (Llama)',
    inputs=[input_text, turns_slider],
    outputs=output_markdown,
    examples=examples,
    flagging_mode='never'
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
